# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `Courses` から **受講生が5人以上いる class を求める**
* 入力 DF: `Courses(student: str, class: str)`
* 出力: 列は **`class` のみ**（順序任意・重複なし）

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → `groupby.size()` → 閾値抽出 → 最終投影。
> 並び順要件なしのため **ソートは行わない**。

```python
import pandas as pd

def classes_with_at_least_five_students(Courses: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['class']
    """
    # 列最小化して集約（pandas 2.2.2 の groupby.size は列名 'size'）
    agg = (
        Courses[['class']]
        .groupby('class', as_index=False)
        .size()
    )
    # 5人以上だけを残し、仕様列のみ返す
    out = agg.loc[agg['size'] >= 5, ['class']].reset_index(drop=True)
    return out

Analyze Complexity
Runtime 287 ms
Beats 34.25%
Memory 68.07 MB
Beats 65.28%

```

## 3) アルゴリズム説明

* 使ったAPI

  * `DataFrame.groupby('class', as_index=False).size()`：クラスごとの件数を計算（`sort` せず、余計な列を持たない）
  * `DataFrame.loc[...]`：件数の閾値（5）でフィルタ
  * `reset_index(drop=True)`：返却 DF を連番インデックスに整える（列順は `['class']` のみ）
* **NULL / 重複 / 型**

  * 入力は `(student, class)` が主キー相当の前提だが、実装は `class` の件数を素直に数えるため、重複があればそのまま件数に反映
  * `class` が `NaN` の行が存在する場合でも `groupby` は `NaN` をグループ化対象外にしない（pandas は `NaN` をキーとしては別扱いしないため、`NaN` を含めたカウントが不要なら事前に `dropna(subset=['class'])` を行う）

## 4) 計算量（概算）

* `groupby.size()`：**O(N)**（ハッシュ集約想定、グループ数を G とするとメモリは O(G)）
* フィルタと最終投影：**O(G)**

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム Courses]
  B[前処理 列最小化 class のみ]
  C[groupby.size で件数集計]
  D[件数が5以上を抽出]
  E[出力 class 列のみ]
  A --> B
  B --> C
  C --> D
  D --> E
```

いい感じの数値ですが、**まだ短縮の余地あり**です。ポイントは「**中間 DataFrame を作らない**」「**ソートしない**」「（可能なら）**カテゴリ化で bin count**」です。

---

## まずは“軽い差分”で速くする

`groupby(...).size()` は中間 DF（`size` 列付き）を作るのでコスト増です。
**Series で完結する `value_counts(sort=False)` → 閾値抽出**に替えると、メモリアロケーションが減って速くなりやすいです。

```python
import pandas as pd

def classes_with_at_least_five_students(Courses: pd.DataFrame) -> pd.DataFrame:
    # 中間DFを作らず Series→Series のまま集計（ソートもしない）
    cnt = Courses['class'].value_counts(sort=False)  # dropna=True が既定
    keep = cnt.index[cnt >= 5]
    return pd.DataFrame({'class': keep}).reset_index(drop=True)

Analyze Complexity
Runtime 248 ms
Beats 94.30%
Memory 67.26 MB
Beats 99.57%

```

* 並び順要件なし ⇒ **sort=False** は必須（既定は True で無駄に並び替えます）
* `value_counts` は `groupby.size` より**割り当てが少なめ**で速いことが多いです

---

## さらに攻める（大規模・繰り返し呼び出し向け）

### 1) `category` による **bin counting**

文字列キーが巨大なときは **カテゴリ化**して整数コードで `np.bincount`。
速度・メモリの**両方**で有利です。

```python
import numpy as np
import pandas as pd

def classes_with_at_least_five_students(Courses: pd.DataFrame) -> pd.DataFrame:
    cat = Courses['class'].astype('category')            # 使い回すなら外で一度だけ!
    codes = cat.cat.codes.to_numpy()
    mask = codes != -1                                   # NaN を除外
    counts = np.bincount(codes[mask],
                         minlength=len(cat.cat.categories))
    classes = cat.cat.categories[counts >= 5]
    return pd.DataFrame({'class': classes}).reset_index(drop=True)

Analyze Complexity
Runtime 259 ms
Beats 80.86%
Memory 67.72 MB
Beats 93.30%

```

**いつ効く？**

* クラスの種類が多い / 文字列が長い
* 同種の集計を**何度も**行う（`astype('category')` を一度だけにできる）

### 2) 最小コピーでの NumPy 直叩き

前処理なしで純粋に速さ重視なら `np.unique`。
（順序要件なしなのでユニークの**ソート**は問題になりません）

```python
import numpy as np
import pandas as pd

def classes_with_at_least_five_students(Courses: pd.DataFrame) -> pd.DataFrame:
    u, c = np.unique(Courses['class'].to_numpy(), return_counts=True)
    return pd.DataFrame({'class': u[c >= 5]}).reset_index(drop=True)

Analyze Complexity
Runtime 254 ms
Beats 88.03%
Memory 67.20 MB
Beats 99.57%

```

> 注意: `np.unique` は `NaN` も一意値として扱います（`NaN` を除外したい場合は事前に `dropna`）。

---

## 実務メモ（さらに数％を取りにいく）

* **NaN の方針を固定**：`value_counts` は既定で NaN を数えません。仕様に合わせて `dropna(subset=['class'])` or `dropna=False` を明示。
* **再利用できるなら category をキャッシュ**：同じ DF で何度も集計する場合、`Courses['class']` を最初から `category` にしておくのが最強。
* **I/O・print・不要な `sort_values` は厳禁**（既に守れているのでOK）。
* **メモリ 68MB 改善**：`class` を `category` にするとメモリが目に見えて下がります（辞書 + コードの二層表現）。

---

## まとめ（おすすめ順）

1. **`value_counts(sort=False)` 版**（最小変更・高コスパ）
2. 文字列が巨大 or 繰り返し集計 → **`category` + `np.bincount`**
3. とにかく最短経路 → **`np.unique(..., return_counts=True)`**

この順で試すと、**287ms → 二桁% 改善**が出るケースが多いです。
